# Test

In [1]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
import itertools

# Icon mapping per sub_category
icon_map = {
    "Turismo Termal y Balnearios": ("spa", "green"),
    "Restaurantes": ("utensils", "blue"),
    "Agencias de viaje y guías": ("suitcase", "purple"),
    "Transporte turístico": ("bus", "orange"),
    "Culturales": ("landmark", "cadetblue"),
    "Tratamientos y terapias de relajación": ("heartbeat", "pink"),
    "Terapias alternativas y holísticas": ("leaf", "darkgreen"),
    "Espiritual y Ancestral": ("place-of-worship", "lightgray"),
    "Retiros": ("peace", "darkblue"),
    "Salud": ("medkit", "lightred"),
    "Financieros": ("money-bill", "gray"),
    "Transporte Público": ("subway", "lightblue"),
    "Peajes": ("road", "lightgray"),
    "Estaciones de servicio": ("gas-pump", "black"),
    "Naturaleza y Montaña": ("mountain", "darkgreen"),
    "Naturaleza & Montaña": ("mountain", "darkgreen"),
    "Caficultura": ("coffee", "brown"),
    "Artesanías y oficios": ("palette", "orange"),
    "Agroturismo": ("tractor", "green"),
    "Cultura y Comunidad": ("users", "purple"),
    "Alojamiento": ("bed", "darkred"),
    "Naturales": ("tree", "green"),
    "Arqueología y fósiles": ("university", "gray"),
    "Patrimonio colonial": ("archway", "black"),
    "Cultura ancestral": ("book", "darkpurple"),
    "Biodiversidad y selva": ("leaf", "darkgreen"),
    "Arquitectura simbólica": ("building", "gray"),
    "Naturaleza Extrema": ("bolt", "red")
}

# Load your cleaned dataset
#@st.cache_data
def load_data():
    return pd.read_csv("datain/map_data.csv")
def load_data_termales():
    return pd.read_csv("datain/termales_priorizados.csv", encoding='ISO-8859-1')

# Scraped data
df = load_data()

# Replacing NAs
df['average_rating'] = df['average_rating'].astype(str)
df['average_rating'] = df['average_rating'].fillna("No Info")

df['user_ratings_total'] = df['user_ratings_total'].fillna(0)
df['user_ratings_total'] = df['user_ratings_total'].astype(int)

# Prioritized termales
prioritized_df = load_data_termales()

# Split lat/lon from 'Georreferenciación'
lat_lon_split = prioritized_df["Georreferenciación"].str.split(",", expand=True)
prioritized_df["latitude"] = lat_lon_split[0].astype(float)
prioritized_df["longitude"] = lat_lon_split[1].astype(float)

# Verify structure after transformation
prioritized_df = prioritized_df[["Centro Termal", "Municipio", "Priorizado", "latitude", "longitude"]]
prioritized_df = prioritized_df.dropna(subset=['latitude'])



2025-07-14 16:27:13.919 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
df.columns

Index(['Unnamed: 0', 'corredor', 'cod_mpio', 'departamento', 'cod_dpto',
       'municipio', 'info_type', 'category', 'sub_category', 'keyword',
       'place_id', 'name', 'types', 'latitude', 'longitude', 'address',
       'average_rating', 'user_ratings_total', 'place_link', 'phone',
       'website', 'opening_hours', 'review_1_rating', 'review_1_text',
       'review_2_rating', 'review_2_text', 'review_3_rating', 'review_3_text',
       'review_4_rating', 'review_4_text', 'review_5_rating', 'review_5_text',
       'city'],
      dtype='object')

In [59]:

# Sidebar filter
all_categories = sorted(df['sub_category'].unique())
selected_categories = all_categories[0:3]


# Filtered data
filtered_df = df[df['sub_category'].isin(selected_categories)]

print(filtered_df.columns)

# Assign colors to each subcategory
colors = itertools.cycle(["blue", "green", "red", "orange", "purple", "darkred", "cadetblue", "pink"])
color_map = {cat: next(colors) for cat in all_categories}

# Center of the map
center_lat = filtered_df['latitude'].mean()
center_lon = filtered_df['longitude'].mean()

# Create folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# Add markers
for _, row in filtered_df.iterrows():
    icon_name, color = icon_map.get(row['sub_category'], ("map-marker", "gray"))
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"""
            <b>{row['name']}</b><br>
            Municipio: {row['municipio']}<br>
            Tipo de lugar: {row['sub_category']}<br>
            Tipo: {row['types']}<br>
            Calificación promedio: {row['average_rating']} ({int(row['user_ratings_total'])} reviews)
        """,
        icon=folium.Icon(icon=icon_name, color=color, prefix="fa")
    ).add_to(m)

# Hot Springs
for _, row in prioritized_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"""
            <b>{row['Centro Termal']}</b><br>
            Municipio: {row['Municipio']}<br>
            Priorizado: {row['Priorizado']}
        """,
        icon=folium.Icon(color='darkpurple', icon='water', prefix='fa')
    ).add_to(m)

m

Index(['Unnamed: 0', 'corredor', 'cod_mpio', 'departamento', 'cod_dpto',
       'municipio', 'info_type', 'category', 'sub_category', 'keyword',
       'place_id', 'name', 'types', 'latitude', 'longitude', 'address',
       'average_rating', 'user_ratings_total', 'place_link', 'phone',
       'website', 'opening_hours', 'review_1_rating', 'review_1_text',
       'review_2_rating', 'review_2_text', 'review_3_rating', 'review_3_text',
       'review_4_rating', 'review_4_text', 'review_5_rating', 'review_5_text',
       'city'],
      dtype='object')
